# ADVANCED OBJECTS

First connect to the database 'DS2':

In [ ]:
-- connection: host='localhost' dbname='ds2' user='ds2' 

------------------------------------------------------------------------------------------------------------

# VIEW

Create a view 'view_custSafe' not displaying credit card and gender information of customers:

In [ ]:
drop view if exists view_custSafe;
CREATE VIEW view_custSafe as 
SELECT customerid, firstname, lastname, address1, address2, city, state, zip, country, region, email, phone, username, password, age, income
FROM customers;

Retreive data from customer no 5432 trough this view

In [ ]:
select * from view_custsafe where customerid=5432;

Create the materialized view 'view_custSafe_mtz' with same properties and with data:

In [ ]:
drop view if exists view_custSafe_mtz;
CREATE MATERIALIZED VIEW view_custSafe_mtz as 
SELECT customerid, firstname, lastname, address1, address2, city, state, zip, country, region, email, phone, username, password, age, income
FROM customers 
WITH DATA;

Retreive data from customer no 5432 trough this view

In [ ]:
select * from view_custsafe_mtz where customerid=5432;

Delete the line of customer no 5432 in table 'customers':

In [ ]:
delete from customers where customerid=5432;

rerun above 'select' transactions  on both views:

In [ ]:
select * from view_custsafe where customerid=5432;
select * from view_custsafe_mtz where customerid=5432;


Why ? Type the command to fix these:

In [ ]:
REFRESH MATERIALIZED VIEW view_custsafe_mtz;


Create a view 'view_custOrdersItem' displaying all items attached to the customerid 11769 ?
Note that the view should contain following informations : customerid,firstname,lastname,orderid,totalamount,prodid,quantity 

In [ ]:
drop view if exists view_custOrdersItem;
create view view_custOrdersItem as select c.customerid,firstname,lastname,o.orderid,totalamount,prod_id,quantity from customers c
inner join orders o on (o.customerid=c.customerid)
inner join orderlines ol on (o.orderid = ol.orderid) where c.customerid=11769; 

Your manager would like to have same informations as view_custOrdersItem but with the name of products (products.title).
How would you proceed ?

In [ ]:
drop view if exists view_custOrdersItemProduct;
select title, vCOI.* from view_custOrdersItem vCOI
inner join products p on (p.prod_id=vCOI.prod_id);

------------------------------------------------------------------------------------------------------------

# INDEX

__You can prefix your statement with keywords '_EXPLAIN ANALYZE_' to display execution stats'__

Create the following table and measure the time:

In [ ]:
SET max_parallel_workers_per_gather TO 0;
DROP TABLE IF EXISTS test;
CREATE TABLE test (i integer not null, t text);
EXPLAIN ANALYZE INSERT INTO test SELECT i, md5(i::text) FROM generate_series(1, 10000000) i;


Run a 'select' query to retrieve line with i=990 000

In [ ]:
EXPLAIN ANALYZE SELECT * FROM test WHERE i = 990000;

How much time your query takes ?
How can you improve it ?

In [ ]:
create index on test(i);

In [ ]:
EXPLAIN ANALYZE SELECT * FROM test WHERE i = 990000;

How much time your query takes ?

Now we need to purge data, how we can do it ?

In [ ]:
truncate table test;

Try to reload data and measure time  :

In [ ]:
EXPLAIN ANALYZE INSERT INTO test SELECT i, md5(i::text) FROM generate_series(1, 10000000) i;

Compare the time with and without index, what do you notice ?


------------------------------------------------------------------------------------------------------------

# TRIGGER

In this exercice, we will write a trigger which will catch all deletes / updates done on the product table and trace these operations into the table products_audit.<br/>

Firstly, create a table 'product_audit' recording all modifications on the products table :

In [ ]:
DROP TABLE IF EXISTS products_audit;
CREATE TABLE products_audit
(
    operation         varchar(20)   NOT NULL,
    stamp             timestamp NOT NULL,
    userid            text      NOT NULL,
    prod_id serial NOT NULL,
    category smallint NOT NULL,
    title text NOT NULL,
    actor text NOT NULL,
    price numeric NOT NULL,
    special smallint,
    common_prod_id integer NOT NULL
)

Use the following function nammed 'f_products_audit' to insert data into the products_audit table. <br/>
Note that you can use :
* the now() function to get the current date
* the system variable TG_OP to determine the operation type (DELETE/UPDATE) and OLD to make reference to old values.
* user to get the connected username

In [ ]:
DROP TRIGGER IF EXISTS tr_products_audit on products;
DROP FUNCTION IF EXISTS f_products_audit();
CREATE OR REPLACE FUNCTION f_products_audit() RETURNS TRIGGER
AS $BODY$
    BEGIN
    -- Business logic 
    -- After each update/ delete 
    INSERT INTO products_audit SELECT TG_OP, now(), user, OLD.*;  -- solution
    RETURN NULL;
    END;
$BODY$
LANGUAGE 'plpgsql' ;

Now, create a trigger calling the f_products_audit function after each modification :

In [ ]:
DROP TRIGGER IF EXISTS tr_products_audit on products;
CREATE TRIGGER tr_products_audit
BEFORE INSERT OR UPDATE OR DELETE ON products
    FOR EACH ROW EXECUTE FUNCTION f_products_audit();

Check your trigger by updating the price of a product :

In [ ]:
update products set price= price * 1.2 where title ='ACADEMY BEAR';

Do you see old values in your products_audit table :

In [ ]:
select * from products_audit;

Check your trigger by deleting a product :

In [ ]:
delete from products where prod_id=1970;

Do you see the deleted row :

In [ ]:
select * from products_audit;

------------------------------------------------------------------------------------------------------------

# Prepared Statements

We want to know : "what is the number of DVD and the average price for a given category." For instance, the store has 3211 DVD referenced as 'comedy' for an average price of 20.02 \$.

In [ ]:
SELECT categoryname, count(prod_id), avg(price)
FROM categories,products
WHERE categories.category=products.category AND categoryname='Comedy'
GROUP BY categoryname;

Transform this query in a prepare statement with the category name as a parameter:

In [ ]:
PREPARE infoPerCategory(text) AS
SELECT categoryname, count(prod_id), avg(price)
FROM categories,products
WHERE categories.category=products.category AND categoryname=$1
GROUP BY categoryname;

Execute the prepare statement for categories 'Sci-Fi' and 'Family':

In [ ]:
EXECUTE infoPerCategory('Sci-Fi');

In [ ]:
EXECUTE infoPerCategory('Family');

Clean it up:

In [ ]:
DEALLOCATE ALL

------------------------------------------------------------------------------------------------------------

# Partitioning

We have increased the number of order and according to the trend we foresee a number of orders close to 1 000 000.

Since the main use of table 'orders_management'  is to prepare online reports for management, we found out that most queries will just access the last week's, month's or quarter's data <br/>
Which kind of partitioning, would you recommand and on which column ?

In [ ]:
DROP TABLE IF EXISTS orders_management;
CREATE TABLE orders_management
(
  orderid serial NOT NULL,
  orderdate date NOT NULL,
  customerid integer,
  netamount numeric NOT NULL,
  tax numeric NOT NULL,
  totalamount numeric NOT NULL
) PARTITION BY RANGE (orderdate);


In [ ]:
CREATE TABLE orders_management_y2009m01 PARTITION OF orders_management
    FOR VALUES FROM ('2009-01-01') TO ('2009-02-01');
CREATE TABLE orders_management_y2009m02 PARTITION OF orders_management
    FOR VALUES FROM ('2009-02-01') TO ('2009-03-01');
CREATE TABLE orders_management_y2009m03 PARTITION OF orders_management
    FOR VALUES FROM ('2009-03-01') TO ('2009-04-01');
CREATE TABLE orders_management_y2009m04 PARTITION OF orders_management
    FOR VALUES FROM ('2009-04-01') TO ('2009-05-01');
CREATE TABLE orders_management_y2009m05 PARTITION OF orders_management
    FOR VALUES FROM ('2009-05-01') TO ('2009-06-01');
CREATE TABLE orders_management_y2009m06 PARTITION OF orders_management
    FOR VALUES FROM ('2009-06-01') TO ('2009-07-01');
CREATE TABLE orders_management_y2009m07 PARTITION OF orders_management
    FOR VALUES FROM ('2009-07-01') TO ('2009-08-01');
CREATE TABLE orders_management_y2009m08 PARTITION OF orders_management
    FOR VALUES FROM ('2009-08-01') TO ('2009-09-01');
CREATE TABLE orders_management_y2009m09 PARTITION OF orders_management
    FOR VALUES FROM ('2009-09-01') TO ('2009-10-01');
CREATE TABLE orders_management_y2009m10 PARTITION OF orders_management
    FOR VALUES FROM ('2009-10-01') TO ('2009-11-01');
CREATE TABLE orders_management_y2009m11 PARTITION OF orders_management
    FOR VALUES FROM ('2009-11-01') TO ('2009-12-01');
CREATE TABLE orders_management_y2009m12 PARTITION OF orders_management
    FOR VALUES FROM ('2009-12-01') TO ('2010-01-01');

Are you ready to load data ? so let's go !

In [ ]:
insert into orders_management select * from orders ;

To make sure, we are using partitioning if we run a query impacting only one partition then the execution plan should indicate us which partition is involved.<br/>
We can get this information with the command analyse which will be detailed in the optimization section.

In [ ]:
explain analyze  select count(*) from orders_management where orderdate < '2009-01-02';

what you could suggest to get a better response time ?

In [ ]:
create index orders_management_idx on orders_management(orderdate);

In [ ]:
SELECT * from pg_indexes where  tablename='orders_management_y2009m09';

In [ ]:
explain analyze  select count(*) from orders_management where orderdate < '2009-01-02';

Is the response time better ?

Now, it's time to make some cleanup. <br/>
We wish to drop old data before june. How could you do that ?<br/>


In [ ]:
ALTER TABLE IF EXISTS orders_management DETACH PARTITION orders_management_y2009m06;
DROP TABLE orders_management_y2009m06;

Is it slow ? <br/>
If you should make clean up on a basic table like table 'orders', would it be simple to do it ? and why ? Try it:

In [ ]:
DELETE FROM orders
WHERE orderdate >= '2009-06-01' AND orderdate < '2009-07-01';